<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd # 데이터 전처리를 위한 라이브러리
import numpy as np # 행렬 처리를 위한 라이브러리

In [2]:
# Define Number and Level
levels = ['광역시도' ,'시군구' ,'읍면동']
level = '광역시도'
num = 800 # 제한하는 값이 따로 있는지
filename = '19년 7월 인구현황(8월작성)'
grouping = True

# levels에서 선택된 level은 제외
levels.remove(level)

In [3]:
# Load Data
df = pd.read_excel("데이터/{}.xlsx".format(filename), sheet_name=1)
df = df[df.구분==level]
df = df.drop(levels, axis=1)

In [4]:
df.head()

,구분,광역시도,남 19-29세,남 30대,남 40대,남 50대,남 60세 이상,여 19-29세,여 30대,여 40대,여 50대,여 60세이상,남 합계,여 합계,총계
0,광역시도,서울특별시,772628,763229,778853,758124,967381,796778,763233,791426,790462,1170959,4040215,4312858,8353073
450,광역시도,부산광역시,250679,226701,261279,284872,402061,227315,217102,258866,304155,494673,1425592,1502111,2927703
673,광역시도,대구광역시,193178,160362,195810,211316,242479,165759,151258,200962,220150,305662,1003145,1043791,2046936
823,광역시도,인천광역시,232021,219865,250219,259402,265413,212114,206515,240572,258497,314533,1226920,1232231,2459151
991,광역시도,광주광역시,119834,99414,121881,115456,124248,109205,97011,123592,117854,156606,580833,604268,1185101


In [5]:
if grouping:
    # 그룹화 리스트
    gp_name_lst = [('경기도','인천광역시'),('대전광역시','세종특별자치시','충청남도','충청북도'),
                   ('광주광역시','전라남도','전라북도'),('대구광역시','경상북도'),
                   ('부산광역시','울산광역시','경상남도'),('강원도','제주특별자치도')]
    # 그룹화할 이름을 반복문으로 처리
    for gp_names in gp_name_lst:
        # 충청남도와 세종특별자치시만 추출후 합계
        new = df[df.광역시도.isin(gp_names)].sum(axis=0)
        # 충남/세종 합계 새로운 행으로 추가
        df = df.append(new, ignore_index=True)
        # 이름 변경
        df.iloc[-1,0] = '광역시도'
        df.iloc[-1,1] = '/'.join(gp_names) # /으로 지역들을 묶어줌
        # 충남, 세종 제외
        df = df[~df.광역시도.isin(gp_names)]
else:
    # 그룹화할 이름
    gp_names =['충청남도','세종특별자치시']
    # 충청남도와 세종특별자치시만 추출후 합계
    new = df[df.광역시도.isin(gp_names)].sum(axis=0)
    # 충남/세종 합계 새로운 행으로 추가
    df = df.append(new, ignore_index=True)
    # 이름 변경
    df.iloc[-1,0] = '광역시도'
    df.iloc[-1,1] = '충청남도/세종특별자치시'
    # 충남, 세종 제외
    df = df[~df.광역시도.isin(gp_names)]

In [6]:
df

,구분,광역시도,남 19-29세,남 30대,남 40대,남 50대,남 60세 이상,여 19-29세,여 30대,여 40대,여 50대,여 60세이상,남 합계,여 합계,총계
0,광역시도,서울특별시,772628,763229,778853,758124,967381,796778,763233,791426,790462,1170959,4040215,4312858,8353073
3,광역시도,경기도/인천광역시,1245835,1212175,1410536,1369007,1385210,1128124,1150155,1364834,1334668,1634471,6622763,6612252,13235015
4,광역시도,대전광역시/세종특별자치시/충청남도/충청북도,412198,392508,456785,463171,566927,351482,357503,423776,439011,692197,2291589,2263969,4555558
5,광역시도,광주광역시/전라남도/전라북도,371526,307742,408898,442186,586247,322472,289049,380961,415654,756175,2116599,2164311,4280910
6,광역시도,대구광역시/경상북도,375842,326161,401575,452286,575799,309604,297849,388876,447751,728022,2131663,2172102,4303765
7,광역시도,부산광역시/울산광역시/경상남도,580412,536804,639286,691068,849068,494544,500851,618201,694038,1043781,3296638,3351415,6648053
8,광역시도,강원도/제주특별자치도,158801,133769,177796,196504,256094,131470,125034,165677,186133,314668,922964,922982,1845946


In [7]:
# Define Features
male_cols = ['남 19-29세', '남 30대', '남 40대', '남 50대', '남 60세 이상']
female_cols = ['여 19-29세', '여 30대', '여 40대', '여 50대', '여 60세이상']
total_cols = male_cols + female_cols

In [8]:
# Total Population
total_pop = df[total_cols].sum().sum()
print('전체 인구수: ',total_pop)

전체 인구수:  43222320


In [9]:
# 2단계 반올림 전
before_df = df.copy()
# 각 셀값을 전체 인구로 나누고 정해진 값으로 곱ㅎ
before_df[total_cols] = (df[total_cols] / total_pop) * num 
# 남/여/총 합계 구하기
before_df['남 합계'] = before_df[male_cols].sum(axis=1)
before_df['여 합계'] = before_df[female_cols].sum(axis=1)
before_df['총계'] = before_df[['남 합계' ,'여 합계']].sum(axis=1)
# 2단계 남여 각각 합계의 반올림
before_sex_sum = before_df[['남 합계' ,'여 합계']].sum().round()

In [10]:
before_df.head()

,구분,광역시도,남 19-29세,남 30대,남 40대,남 50대,남 60세 이상,여 19-29세,여 30대,여 40대,여 50대,여 60세이상,남 합계,여 합계,총계
0,광역시도,서울특별시,14.300537,14.126572,14.415756,14.032083,17.905212,14.747529,14.126646,14.648469,14.630626,21.673228,74.780160,79.826497,154.606657
3,광역시도,경기도/인천광역시,23.059105,22.436093,26.107548,25.338890,25.638790,20.880397,21.288168,25.261652,24.703311,30.252351,122.580426,122.385878,244.966304
4,광역시도,대전광역시/세종특별자치시/충청남도/충청북도,7.629354,7.264913,8.454613,8.572811,10.493227,6.505565,6.617007,7.843651,8.125635,12.811844,42.414919,41.903702,84.318621
5,광역시도,광주광역시/전라남도/전라북도,6.876558,5.695983,7.568275,8.184401,10.850820,5.968620,5.349995,7.051190,7.693321,13.996009,39.176037,40.059136,79.235173
6,광역시도,대구광역시/경상북도,6.956443,6.036899,7.432734,8.371341,10.657438,5.730447,5.512874,7.197689,8.287403,13.474927,39.454856,40.203339,79.658195


In [11]:
before_sex_sum

남 합계    397.0
여 합계    403.0
dtype: float64

In [12]:
# 3단계 반올림 후
after_df = df.copy()
# 각 셀값을 전체 인구로 나누고 정해진 값으로 곱ㅎ
after_df[total_cols] = (df[total_cols] / total_pop) * num 
# 각 셀을 반올림
after_df[total_cols] = after_df[total_cols].round() 
# 남/여/총 합계 구하기
after_df['남 합계'] = after_df[male_cols].sum(axis=1)
after_df['여 합계'] = after_df[female_cols].sum(axis=1)
after_df['총계'] = after_df[['남 합계' ,'여 합계']].sum(axis=1)
# 3단계 남여 각각 합계의 반올림
after_sex_sum = after_df[['남 합계' ,'여 합계']].sum()

In [13]:
after_df.head()

,구분,광역시도,남 19-29세,남 30대,남 40대,남 50대,남 60세 이상,여 19-29세,여 30대,여 40대,여 50대,여 60세이상,남 합계,여 합계,총계
0,광역시도,서울특별시,14.0,14.0,14.0,14.0,18.0,15.0,14.0,15.0,15.0,22.0,74.0,81.0,155.0
3,광역시도,경기도/인천광역시,23.0,22.0,26.0,25.0,26.0,21.0,21.0,25.0,25.0,30.0,122.0,122.0,244.0
4,광역시도,대전광역시/세종특별자치시/충청남도/충청북도,8.0,7.0,8.0,9.0,10.0,7.0,7.0,8.0,8.0,13.0,42.0,43.0,85.0
5,광역시도,광주광역시/전라남도/전라북도,7.0,6.0,8.0,8.0,11.0,6.0,5.0,7.0,8.0,14.0,40.0,40.0,80.0
6,광역시도,대구광역시/경상북도,7.0,6.0,7.0,8.0,11.0,6.0,6.0,7.0,8.0,13.0,39.0,40.0,79.0


In [14]:
after_sex_sum

남 합계    396.0
여 합계    403.0
dtype: float64

In [15]:
# 2,3단계 남여 합계의 차이
'''
차이는 세 가지 경우로 나뉜다: 남여 각각 차이가 1. 0이거나 / 2. 0보다 크거나 / 3. 0보다 작거나 
1. 0인 경우는 추가적인 일 없이 표 완성
2. 만약 차이가 0보다 큰 경우 : xx.5 보다 작고 xx.5에 가장 가까운 값인 반올림 값에 + 1 
- Why? 반올림하여 내림이 된 값 중 가장 올림에 가까운 값에 1을 더하는 것이 이상적이기 때문
    ex) 2.49999 -> round(2.49999) + 1
3. 만약 차이가 0보다 작은 경우 : xx.5 이상이고 xx.5에 가장 가까운 값인 반올림 값에 - 1
- Why? 반올림하여 올림이 된 값 중 가장 내림에 가까운 값에 1을 빼는 것이 이상적이기 때문
    ex) 2.50001 -> round(2.50001) - 1
'''
sex_diff = before_sex_sum - after_sex_sum

In [16]:
sex_diff

남 합계    1.0
여 합계    0.0
dtype: float64

In [17]:
# 성별 합계 차이를 매꾸는 단계
sex_cols_lst = [male_cols, female_cols]
sex_idx = ['남 합계' ,'여 합계']
print('sex_cols_lst: ',sex_cols_lst)
print('sex_cols: ',sex_idx)

sex_cols_lst:  [['남 19-29세', '남 30대', '남 40대', '남 50대', '남 60세 이상'], ['여 19-29세', '여 30대', '여 40대', '여 50대', '여 60세이상']]
sex_cols:  ['남 합계', '여 합계']


In [18]:
# 차이가 0보다 큰 경우
'''
1단계: 2단계 반올림 전 값을 모두 내림 한 후 0.5를 더 한다.
2단계: 1번에서 한 값과 2단계 반올림 전 값을 뺀다.
3단계: 음수로 나오는 값은 모두 1로 변환. 1이 가장 큰 값이기 때문.
ex) 13.45 -> 13 으로 내림 후 (13 + 0.5) - 13.45 = 0.05
'''
temp = (before_df[sex_cols_lst[0]].astype(int) + 0.5) - before_df[sex_cols_lst[0]] # 1,2번
temp = temp[temp >0].fillna(1) # 3번
v = 1

In [19]:
# 2단계 값
before_df[sex_cols_lst[0]].head()

,남 19-29세,남 30대,남 40대,남 50대,남 60세 이상
0,14.300537,14.126572,14.415756,14.032083,17.905212
3,23.059105,22.436093,26.107548,25.338890,25.638790
4,7.629354,7.264913,8.454613,8.572811,10.493227
5,6.876558,5.695983,7.568275,8.184401,10.850820
6,6.956443,6.036899,7.432734,8.371341,10.657438


In [20]:
# 전체 소수점을 내림한다.
before_df[sex_cols_lst[0]].astype(int).head()

,남 19-29세,남 30대,남 40대,남 50대,남 60세 이상
0,14,14,14,14,17
3,23,22,26,25,25
4,7,7,8,8,10
5,6,5,7,8,10
6,6,6,7,8,10


In [21]:
# 내림된 값에다 0.5를 더한다.
(before_df[sex_cols_lst[0]].astype(int) + 0.5).head()

,남 19-29세,남 30대,남 40대,남 50대,남 60세 이상
0,14.5,14.5,14.5,14.5,17.5
3,23.5,22.5,26.5,25.5,25.5
4,7.5,7.5,8.5,8.5,10.5
5,6.5,5.5,7.5,8.5,10.5
6,6.5,6.5,7.5,8.5,10.5


In [22]:
# 0.5를 더한 값에 2단계를 뺀다
test = (before_df[sex_cols_lst[0]].astype(int) + 0.5) - before_df[sex_cols_lst[0]]
test.head()

,남 19-29세,남 30대,남 40대,남 50대,남 60세 이상
0,0.199463,0.373428,0.084244,0.467917,-0.405212
3,0.440895,0.063907,0.392452,0.161110,-0.138790
4,-0.129354,0.235087,0.045387,-0.072811,0.006773
5,-0.376558,-0.195983,-0.068275,0.315599,-0.350820
6,-0.456443,0.463101,0.067266,0.128659,-0.157438


In [23]:
# 0보다 큰 값만 사용한다.
test[test>0].head()

,남 19-29세,남 30대,남 40대,남 50대,남 60세 이상
0,0.199463,0.373428,0.084244,0.467917,NaN
3,0.440895,0.063907,0.392452,0.161110,NaN
4,NaN,0.235087,0.045387,NaN,0.006773
5,NaN,NaN,NaN,0.315599,NaN
6,NaN,0.463101,0.067266,0.128659,NaN


In [24]:
# NaN은 Not a Number의 약자. 결측치라는 얘기. 
# 결측치는 모두 1로 처리 가장 크기 때문
test2 = test[test>0].fillna(1)
test2.head()

,남 19-29세,남 30대,남 40대,남 50대,남 60세 이상
0,0.199463,0.373428,0.084244,0.467917,1.000000
3,0.440895,0.063907,0.392452,0.161110,1.000000
4,1.000000,0.235087,0.045387,1.000000,0.006773
5,1.000000,1.000000,1.000000,0.315599,1.000000
6,1.000000,0.463101,0.067266,0.128659,1.000000


In [25]:
# 실제합계와의 차이: 절대값을 통해서 음수를 변환하고 정수로 타입을 변환
cnt = int(abs(sex_diff.loc[sex_idx[0]]))
cnt

1

In [26]:
# 1행으로 변환
test3 = test2.values
test3.ravel()

array([0.19946269, 0.37342836, 0.08424444, 0.46791658, 1.        ,
       0.44089535, 0.0639068 , 0.39245186, 0.16111028, 1.        ,
       1.        , 0.23508687, 0.04538674, 1.        , 0.00677335,
       1.        , 1.        , 1.        , 0.31559898, 1.        ,
       1.        , 0.46310055, 0.06726617, 0.12865853, 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 0.02407552, 0.20918174, 1.        , 1.        ])

In [27]:
# 작은 값을 기준으로 각 값들의 위치를 표시하여 정렬
np.argsort(test3.ravel())

array([14, 31, 12,  6, 22,  2, 23,  8,  0, 32, 11, 18,  1,  7,  5, 21,  3,
       30, 29, 28, 27, 26, 25, 24, 17, 19, 33, 16, 15, 13, 10,  9,  4, 20,
       34], dtype=int64)

In [28]:
# 상위 n개 선택
np.argsort(test3.ravel())[:cnt]

array([14], dtype=int64)

In [30]:
# unravel_index라는 함수를 통해서 각 위치 좌표를 추출
row_col = np.unravel_index(np.argsort(test3.ravel())[:cnt], test3.shape)
rows = row_col[0]
cols = row_col[1]
print('rows: ',rows)
print('cols: ',cols)

rows:  [2]
cols:  [4]


In [31]:
# 남자들 인구만 추출
temp = before_df[sex_cols_lst[0]].copy() 
temp.head()

,남 19-29세,남 30대,남 40대,남 50대,남 60세 이상
0,14.300537,14.126572,14.415756,14.032083,17.905212
3,23.059105,22.436093,26.107548,25.338890,25.638790
4,7.629354,7.264913,8.454613,8.572811,10.493227
5,6.876558,5.695983,7.568275,8.184401,10.850820
6,6.956443,6.036899,7.432734,8.371341,10.657438


In [32]:
# 아슬아슬한애가 누구인지 확인
value = temp.iloc[rows[0] ,cols[0]]
print('{}의 {}: {}'.format(df[level].iloc[rows[0]], sex_cols_lst[0][cols[0]], value))

대전광역시/세종특별자치시/충청남도/충청북도의 남 60세 이상: 10.493226647713497


In [33]:
# 아슬아슬한 애들에게 1만큼 더하기
temp = after_df[sex_cols_lst[0]].copy()
print('1만큼 더해진 값: ',temp.iloc[rows[0] ,cols[0]] + v)

1만큼 더해진 값:  11.0


In [34]:
# 3단계 데이터프레임에 전라남도의 남 60세 이상에 1을 더해줌
temp.iloc[rows[0] ,cols[0]] = temp.iloc[rows[0] ,cols[0]] + v
temp.head(14)

,남 19-29세,남 30대,남 40대,남 50대,남 60세 이상
0,14.0,14.0,14.0,14.0,18.0
3,23.0,22.0,26.0,25.0,26.0
4,8.0,7.0,8.0,9.0,11.0
5,7.0,6.0,8.0,8.0,11.0
6,7.0,6.0,7.0,8.0,11.0
7,11.0,10.0,12.0,13.0,16.0
8,3.0,2.0,3.0,4.0,5.0


In [35]:
# 위에 내용을 일반화
for r in range(len(rows)):
    temp = before_df[sex_cols_lst[0]].copy()
    value = temp.iloc[rows[r] ,cols[r]]
    print('{}의 {}: {}'.format(df[level].iloc[rows[r]], sex_cols_lst[0][cols[r]], value))
    temp = after_df[sex_cols_lst[0]].copy()
    temp.iloc[rows[r] ,cols[r]] = temp.iloc[rows[r] ,cols[r]] + v
    after_df[sex_cols_lst[0]] = temp
print()

대전광역시/세종특별자치시/충청남도/충청북도의 남 60세 이상: 10.493226647713497



In [36]:
# 부족한 부분이 채워졌으면 합계 계산
after_df['남 합계'] = after_df[male_cols].sum(axis=1)
after_df['여 합계'] = after_df[female_cols].sum(axis=1)
after_df['총계'] = after_df[['남 합계','여 합계']].sum(axis=1)
after_df[['남 합계','여 합계']].sum()

남 합계    397.0
여 합계    403.0
dtype: float64

In [37]:
after_df

,구분,광역시도,남 19-29세,남 30대,남 40대,남 50대,남 60세 이상,여 19-29세,여 30대,여 40대,여 50대,여 60세이상,남 합계,여 합계,총계
0,광역시도,서울특별시,14.0,14.0,14.0,14.0,18.0,15.0,14.0,15.0,15.0,22.0,74.0,81.0,155.0
3,광역시도,경기도/인천광역시,23.0,22.0,26.0,25.0,26.0,21.0,21.0,25.0,25.0,30.0,122.0,122.0,244.0
4,광역시도,대전광역시/세종특별자치시/충청남도/충청북도,8.0,7.0,8.0,9.0,11.0,7.0,7.0,8.0,8.0,13.0,43.0,43.0,86.0
5,광역시도,광주광역시/전라남도/전라북도,7.0,6.0,8.0,8.0,11.0,6.0,5.0,7.0,8.0,14.0,40.0,40.0,80.0
6,광역시도,대구광역시/경상북도,7.0,6.0,7.0,8.0,11.0,6.0,6.0,7.0,8.0,13.0,39.0,40.0,79.0
7,광역시도,부산광역시/울산광역시/경상남도,11.0,10.0,12.0,13.0,16.0,9.0,9.0,11.0,13.0,19.0,62.0,61.0,123.0
8,광역시도,강원도/제주특별자치도,3.0,2.0,3.0,4.0,5.0,2.0,2.0,3.0,3.0,6.0,17.0,16.0,33.0
